
# AI-агент для генерации AutoTRIZ

Основан на


In [25]:

# === УСТАНОВКА ЗАВИСИМОСТЕЙ (Colab) ===
!pip install gigachat langgraph langchain_gigachat langchain_tavily -q -U


In [40]:

# === CONFIG ===
import os
from google.colab import userdata

# Провайдер: 'gigachat' или 'openai'
PROVIDER = 'gigachat'

OPENAI_MODEL = 'gpt-4o-mini'
GIGACHAT_MODEL = 'GigaChat-2-Max'

credentials = None
if PROVIDER == 'openai':
    import getpass
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Введите OPENAI_API_KEY: ')
elif PROVIDER == 'gigachat':
    # Берём ключ из Colab userdata (заранее добавьте в UI: Settings → User data → giga_key)
    credentials = userdata.get('giga_key')
    if not credentials:
        raise RuntimeError("Не найден userdata['giga_key']. Добавьте ключ в Colab (Settings → User data).")
    os.environ["GIGACHAT_BASE_URL"] = "https://gigachat.devices.sberbank.ru/api/v1"

# Опциональный ключ Tavily для поиска (по желанию можно оставить пустым в Settings → User data → tavily_key)
tavily_key = userdata.get('tavily_key')
if tavily_key:
    os.environ['TAVILY_API_KEY'] = tavily_key

MODEL_NAME = OPENAI_MODEL if PROVIDER == 'openai' else GIGACHAT_MODEL
print(f"Провайдер: {PROVIDER}, модель: {MODEL_NAME}")


Провайдер: gigachat, модель: GigaChat-2-Max


## Библиотечные вспомогательные функции

In [34]:

from typing import Optional, Literal
from typing_extensions import TypedDict, Annotated
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser

# Провайдеры
try:
    from langchain_openai import ChatOpenAI
except Exception:
    ChatOpenAI = None

try:
    from langchain_gigachat import GigaChat
except Exception:
    GigaChat = None

# Поиск (через официальный tavily-python)
from langchain_tavily import TavilySearch

def get_llm_from_config(config):
    """Возвращает LLM на основе config['configurable'].
    Для GigaChat отключает SSL verify (verify_ssl_certs=False)."""
    provider = config.get('configurable', {}).get('provider', 'openai')
    model = config.get('configurable', {}).get('model', 'gpt-4o-mini')
    if provider == 'openai':
        if ChatOpenAI is None:
            raise RuntimeError("langchain-openai не установлен")
        return ChatOpenAI(model=model, temperature=0)
    elif provider == 'gigachat':
        if GigaChat is None:
            raise RuntimeError("langchain-gigachat не установлен")
        return GigaChat(
            credentials=credentials,
            model=model,
            profanity_check=False,
            top_p=0,
            timeout=120,
            verify_ssl_certs=False  # 🔑 отключаем проверку SSL
        )
    else:
        raise ValueError(f"Неизвестный провайдер: {provider}")

def tavily_search(query: str, max_results: int = 3) -> str:
    """Возвращает короткое резюме результатов поиска. Если ключа нет — graceful fallback."""
    try:
        key = os.environ.get("TAVILY_API_KEY")
        if not key:
            return f"Поиск недоступен (нет TAVILY_API_KEY). Запрос: {query}"
        client = TavilySearch(api_key=key)
        res = client.search(query=query, search_depth="advanced", max_results=max_results)
        items = res.get('results', [])
        lines = [f"- {it.get('title')}: {it.get('url')} — {it.get('content','')[:200]}..." for it in items]
        return "\n".join(lines) if lines else "Ничего релевантного не найдено."
    except Exception as e:
        return f"Поиск временно недоступен. Запрос: {query}. Ошибка: {e}"


## Состояние, вспомогательные функции и универсальный запрос к LLM

In [35]:

class TRIZGraphState(TypedDict):
    """Состояние TRIZ-агента: T, C, I, S, плюс контекст и фидбек."""
    main_task: Annotated[str, "Основная задача/проблема от пользователя"]
    context_analysis: Optional[Annotated[str, "Анализ контекста и похожих решений"]]
    feedback: Optional[Annotated[str, "Фидбек пользователя"]]
    problem_definition: Optional[Annotated[str, "Четкое определение проблемы"]]
    contradictions: Optional[Annotated[str, "Выявленные противоречия"]]
    inventive_principles: Optional[Annotated[str, "Применимые изобретательские приемы"]]
    solution_concept: Optional[Annotated[str, "Концепция решения на основе ТРИЗ"]]

def state_to_string(state: TRIZGraphState) -> str:
    lines = []
    for field, annotation in TRIZGraphState.__annotations__.items():
        value = state.get(field, "")
        if value:
            desc = ""
            if hasattr(annotation, "__metadata__") and annotation.__metadata__:
                desc = annotation.__metadata__[0]
            lines.append(f"{desc} ({field}): {value}")
    return "\n".join(lines)

def ask_triz_llm(state: TRIZGraphState, question: str, context: str, config) -> str:
    TEMPLATE = """Ты – эксперт по ТРИЗ (теория решения изобретательских задач) и разработке AI-агентов.
Учитывай основную задачу (main_task), текущее состояние анализа и фидбек пользователя (если есть).

Контекст:
{context}

Состояние:
{state}

Вопрос: {question}

Отвечай четко и структурированно, применяя ТРИЗ и приводя обоснования.
"""
    prompt = ChatPromptTemplate.from_messages([("system", TEMPLATE)])
    llm = get_llm_from_config(config)
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"state": state_to_string(state), "question": question, "context": context})


## Узлы графа: T → C → I → S

In [36]:

def problem_definition(state: TRIZGraphState, config):
    context = "Выявляем суть задачи и ограничения."
    result = ask_triz_llm(state, "Сформулируй проблему четко и конкретно. Что именно нужно улучшить или преодолеть?", context, config)
    return {"problem_definition": result}

def identify_contradictions(state: TRIZGraphState, config):
    context = "Определи ключевое противоречие (техническое или физическое)."
    result = ask_triz_llm(state, "Какие два параметра в конфликте? (формат: X vs Y)", context, config)
    return {"contradictions": result}

def apply_inventive_principles(state: TRIZGraphState, config):
    text = (state.get("contradictions", "") or "").lower()
    principles_map = {
        ("скорост", "качеств"): ["Дробление/Сегментация", "Локальное качество"],
        ("простот", "функцион"): ["Посредник", "Вынесение"],
        ("автоном", "контрол"): ["Обратная связь", "Иерархичность"],
        ("персонализ", "универс"): ["Предварительное действие", "Дробление/Сегментация"],
    }
    for (a, b), principles in principles_map.items():
        if a in text and b in text:
            return {"inventive_principles": "Рекомендуемые принципы: " + ", ".join(principles)}
    # fallback к LLM
    context = "Подбери 3–5 принципов из 40 ТРИЗ, объясни, как каждый снимает противоречие."
    question = "Какие принципы применимы и почему?"
    result = ask_triz_llm(state, question, context, config)
    return {"inventive_principles": result}

def generate_solution_concept(state: TRIZGraphState, config):
    context = "Синтез решения на основе выявленного противоречия и выбранных принципов."
    result = ask_triz_llm(state, "Предложи концепцию решения и объясни, как принципы помогают устранить конфликт.", context, config)
    return {"solution_concept": result}


## Оценка качества решения и обратная связь

In [37]:

class QualityAssessment(BaseModel):
    chain_of_thoughts: str = Field(description="Ход рассуждений")
    solution_quality: str = Field(description="Качество решения (оценка)")
    is_satisfactory: bool = Field(description="Удовлетворяет ли решение требованиям?")
    improvement_suggestions: str = Field(description="Рекомендации по улучшению")

def check_solution_quality(state: TRIZGraphState, config) -> Command[Literal["get_feedback", "problem_definition"]]:
    if config.get('configurable', {}).get('skip_search', False):
        return Command(goto="get_feedback")
    search_query = f"{state['problem_definition']} AI agent solution"
    search_results = tavily_search(search_query, max_results=3)

    parser = PydanticOutputParser(pydantic_object=QualityAssessment)
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Проанализируй качество решения по ТРИЗ, сравнив его с найденными аналогами.

Состояние:
{state}

Поиск (резюме):
{search_results}

Дай оценку (новизна, реализуемость, соответствие ТРИЗ, риски) и рекомендации.
{format_instructions}
""")
    ]).partial(format_instructions=parser.get_format_instructions())

    llm = get_llm_from_config(config)
    chain = prompt | llm | parser
    assessment = chain.invoke({"state": state_to_string(state), "search_results": search_results})

    context_analysis = (
        f"Поиск по '{search_query}':\n{search_results}\n\n"
        f"Оценка: {assessment.solution_quality}"
    )
    if assessment.is_satisfactory:
        return Command(update={"context_analysis": context_analysis}, goto="get_feedback")
    else:
        return Command(update={"context_analysis": context_analysis}, goto="problem_definition")

RedirectStep = Literal[
    "problem_definition",
    "identify_contradictions",
    "apply_inventive_principles",
    "generate_solution_concept",
    "__end__",
]

class UserFeedback(BaseModel):
    feedback: str = Field(description="Отзыв пользователя")
    next_step: RedirectStep = Field(description="Этап для доработки")
    is_done: bool = Field(description="Завершить ли анализ?")

def get_feedback(state: TRIZGraphState, config) -> Command[Literal[RedirectStep, END]]:
    if config.get('configurable', {}).get('need_interrupt', True):
        fb_text = interrupt(
            "Оцените решение и укажите, что доработать (или напишите 'Готово').\n"
            "1) Что изменить? 2) С какого этапа перезапустить? 3) Готово?\n"
        )
    else:
        fb_text = "Все устраивает."

    parser = PydanticOutputParser(pydantic_object=UserFeedback)
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Проанализируй отзыв пользователя и реши дальнейшие действия.
Состояние:
{state}

Отзыв:
{feedback}

Определи: 1) что улучшить, 2) стартовый этап, 3) завершать ли анализ.
{format_instructions}
""")
    ]).partial(format_instructions=parser.get_format_instructions())

    llm = get_llm_from_config(config)
    chain = prompt | llm | parser
    result = chain.invoke({"state": state_to_string(state), "feedback": fb_text})

    if result.is_done:
        return Command(update={}, goto=END)
    else:
        return Command(update={"feedback": result.feedback}, goto=result.next_step)


## Сборка графа и утилиты

In [38]:

class TRIZGraphConfig(BaseModel):
    need_interrupt: bool = True
    skip_search: bool = False
    provider: Literal['openai', 'gigachat'] = 'gigachat'
    model: str = MODEL_NAME

def create_triz_agent():
    # ВАЖНО: context_schema вместо устаревшего config_schema
    graph = StateGraph(TRIZGraphState, context_schema=TRIZGraphConfig)
    graph.add_node("problem_definition", problem_definition)
    graph.add_node("identify_contradictions", identify_contradictions)
    graph.add_node("apply_inventive_principles", apply_inventive_principles)
    graph.add_node("generate_solution_concept", generate_solution_concept)
    graph.add_node("check_solution_quality", check_solution_quality)
    graph.add_node("get_feedback", get_feedback)

    graph.add_edge(START, "problem_definition")
    graph.add_edge("problem_definition", "identify_contradictions")
    graph.add_edge("identify_contradictions", "apply_inventive_principles")
    graph.add_edge("apply_inventive_principles", "generate_solution_concept")
    graph.add_edge("generate_solution_concept", "check_solution_quality")

    memory = MemorySaver()
    return graph.compile(checkpointer=memory)

def generate_triz_report(state: TRIZGraphState) -> str:
    return f"""# ОТЧЕТ TRIZ-АНАЛИЗА

## 1. Исходная задача
{state.get('main_task', '—')}

## 2. Определение проблемы
{state.get('problem_definition', '—')}

## 3. Выявленное противоречие
{state.get('contradictions', '—')}

## 4. Изобретательские приемы
{state.get('inventive_principles', '—')}

## 5. Концепция решения
{state.get('solution_concept', '—')}

## 6. Контекстный анализ
{state.get('context_analysis', '—')}

## 7. Фидбек пользователя
{state.get('feedback', '—')}
"""

def collect_state_from_stream(app, inputs, config):
    """Выполняет граф, аккумулирует состояние и печатает узлы."""
    state = {"main_task": inputs["main_task"]}
    for event in app.stream(inputs, config=config):
        if isinstance(event, dict):
            for node, output in event.items():
                if node != "__interrupt__" and isinstance(output, dict):
                    state.update(output)
                    print(f"\n📋 {node}")
                    for k, v in output.items():
                        print(f"  - {k}:\n{v}\n")
    return state


## Демонстрация: неинтерактивный прогон

In [41]:

problem = "Мы хотим ускорить отклик мультиагентной системы, не снижая качество решений."

config = {
    "configurable": {
        "need_interrupt": False,
        "skip_search": True,           # можно True, если не нужен поиск
        "provider": PROVIDER,
        "model": MODEL_NAME,
        "thread_id": "demo-run-1",     # обязательно для MemorySaver
        "checkpoint_ns": "triz-demo"
    }
}

app = create_triz_agent()
state = collect_state_from_stream(app, {"main_task": problem}, config)
print("\n" + "-"*80 + "\nИтоговый отчет:\n")
print(generate_triz_report(state))



📋 problem_definition
  - problem_definition:
**Формулировка проблемы:**  
Необходимо повысить скорость отклика мультиагентной системы, сохранив высокое качество принимаемых решений.

---

### **Анализ ситуации через призму ТРИЗ**
1. **Противоречие**:  
   Чем быстрее система принимает решение, тем ниже вероятность тщательного анализа всех факторов и выбора оптимального варианта. И наоборот — повышение качества требует больше времени на обработку информации и принятие взвешенного решения.
   
2. **Идеальное конечное решение (ИКР)**:  
   Система должна мгновенно принимать качественные решения без увеличения вычислительных ресурсов и снижения точности.

3. **Ограничения и ресурсы**:  
   - Ограничение по ресурсам (вычислительные мощности ограничены).  
   - Необходимость сохранения высокого уровня надежности и устойчивости решений.  
   - Требование минимизации задержек между получением запроса и выдачей результата.

4. **Инструменты ТРИЗ для преодоления противоречия**:  
   - Принцип р


## Демонстрация: интерактивный режим (interrupt/resume)
Выполните ячейку: агент остановится на этапе обратной связи и спросит комментарии.  
Введите ответ прямо в консоли ячейки (например: «Добавить кэширование; начать с противоречий»).  
Чтобы завершить — напишите «Готово».


In [42]:

problem = "Нужно повысить пропускную способность API-инструментов агента без ухудшения надежности."
config = {
    "configurable": {
        "need_interrupt": True,
        "skip_search": False,          # поставьте True, если нет TAVILY_API_KEY
        "provider": PROVIDER,
        "model": MODEL_NAME,
        "thread_id": "demo-run-2",
        "checkpoint_ns": "triz-demo"
    }
}

app = create_triz_agent()
inputs = {"main_task": problem}
print("🔧 Запуск интерактивного анализа...")

for event in app.stream(inputs, config=config):
    if isinstance(event, dict):
        for node, output in event.items():
            if node != "__interrupt__" and isinstance(output, dict):
                print(f"\n📋 {node}")
                for k, v in output.items():
                    print(f"  - {k}:\n{v}\n")

while '__interrupt__' in event:
    prompt_text = event['__interrupt__'][0].value
    print("\n⚠️ Запрос фидбека:\n" + prompt_text)
    user_input = input("Ваш ответ: ")
    for event in app.stream(Command(resume=user_input), config=config):
        if isinstance(event, dict):
            for node, output in event.items():
                if node != "__interrupt__" and isinstance(output, dict):
                    print(f"\n📋 {node}")
                    for k, v in output.items():
                        print(f"  - {k}:\n{v}\n")
print("\nГотово.")


🔧 Запуск интерактивного анализа...

📋 problem_definition
  - problem_definition:
**Формулировка проблемы:**  
Необходимо увеличить количество запросов, обрабатываемых агентом через API-интерфейсы, сохраняя высокую надежность системы и качество обработки каждого запроса.

**Конкретизация требований:**  
1. **Пропускная способность**: Увеличение количества одновременно выполняемых запросов без перегрузки инфраструктуры и снижения производительности.
2. **Надежность**: Сохранение высокой доступности сервиса даже при увеличении нагрузки — минимизация отказов и ошибок.
3. **Качество обработки**: Обеспечение корректности выполнения всех операций независимо от объема поступающих запросов.
4. **Масштабируемость**: Возможность быстрого масштабирования ресурсов при росте числа пользователей и интенсивности работы.
5. **Эффективное использование ресурсов**: Оптимальное распределение вычислительных мощностей между различными компонентами системы.

**Ограничения и противоречия:**  
- Чем больше зап

## Быстрый однопроходный TRIZ-анализ (необязательно)

In [43]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("""Проведи быстрый TRIZ-анализ:
{problem}

Ответь по структуре:
1) Противоречие
2) 3–5 применимых изобретательских приемов
3) Концепция решения
""")
llm = get_llm_from_config({"configurable": {"provider": PROVIDER, "model": MODEL_NAME}})
print((prompt | llm | StrOutputParser()).invoke({"problem": "Оптимизировать стоимость инференса без падения качества"}))


### Анализ по методологии ТРИЗ (Теория Решения Изобретательских Задач)

#### 1. Противоречие  
Необходимо снизить затраты на проведение инференса модели машинного обучения, сохранив высокое качество результатов. Чем больше ресурсов выделяется на вычисления — тем лучше точность прогноза, однако растут расходы. Снижение вычислительных затрат ведет к ухудшению точности предсказания.

**Противоречие:** высокая производительность требует больших расходов → снижение стоимости снижает качество работы модели.

---

#### 2. Применимые приемы:  
- **Принцип дробления**: разделить процесс инференса на несколько этапов, оптимизируя каждый отдельно (например, предварительная фильтрация входных данных).
- **Принцип динамичности**: адаптивная настройка сложности моделей в зависимости от конкретных запросов пользователей (менее требовательные запросы обрабатываются упрощенными моделями).
- **Принцип перехода в другое измерение**: использование распределенных облачных решений или гибридных архитектур (